In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline

from env import *
from states import *
from agents import *
from models import *
from util import *

/Users/hanqiu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hanqiu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hanqiu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/hanqiu/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework

In [2]:
env = Env(Game(),DefaultPlayer(),state_generator=StateExtend)
s,I,cs = env.init((1,1))
display(s.stack,cs)
display(s.state_vec[0,:(2*s.deck_size)].reshape((2,s.deck_size)))
display(I.state_vec[0,:(2*s.deck_size)].reshape((2,s.deck_size)))

[1, 2, 5, 6, 7, 8, 9, 11]

[[0, 8, 14, 15, 16, 17, 18, 19, 21],
 [4, 4, 10, 12, 14, 16, 18, 20, 21],
 [0, 1, 7, 9, 10, 13, 15, 17, 20],
 [2, 3, 3, 5, 6, 11, 12, 13, 19]]

array([[ 8, -1, -1, 11,  9, -1, -1, -1, -1, -1,  9, -1,  9, 10,  8,  8,
         8,  8,  8,  8,  9,  8],
       [10, 10, 11, 11,  9, 11, 11, 10,  8, 10, 10, 11, 11, 11,  9, 10,
         9, 10,  9, 11, 10,  9]])

array([[-2, -1, -1, -2,  9, -1, -1, -1, -1, -1,  9, -1,  9, -2, -2, -2,
        -2, -2, -2, -2,  9, -2],
       [-2, -2, -2, -2,  9, -2, -2, -2, -2, -2, -2, -2, -2, -2,  9, -2,
         9, -2,  9, -2, -2,  9]])

In [3]:
a_base = Agent()
agents = [a_base,a_base,a_base,a_base,]
run_games(N_iter=1,agents=agents,state_generator=StateExtend)

(array([-20]), array([140]))

#### Evaluations

Compare performance of different agents.

In [ ]:
## load models and set up agents
q_model = QModel(save_dir='model/')
q_model.restore('dqn-v3/750000')

a_base = Agent()
a_rl = RlAgent(q_model)
e_sim = Env(Game(),DefaultPlayer(),state_generator=StateExtend)
a_mcts1 = MCTSAgent(q_model,e_sim,N_search=1600)
for a in [a_base,a_rl,a_mcts1]:
    a.infer_flag = True

# agents = [a_base,a_base,a_base,a_base]
# agents = [a_rl,a_base,a_rl,a_base]
# agents = [a_base,a_rl,a_base,a_rl]

agents = [a_mcts1,a_base,a_mcts1,a_base]
# agents = [a_base,a_mcts1,a_base,a_mcts1]
# agents = [a_mcts1,a_rl,a_mcts1,a_rl]
# agents = [a_rl,a_mcts1,a_rl,a_mcts1]

N_iter = 10
# N_iter = 10000

In [ ]:
## simulate for a few games and plot the result
tt = time.time()
curr_scores,eval_scores = \
    run_games(N_iter=N_iter,agents=agents,state_generator=StateExtend)
print(time.time()-tt,np.mean(curr_scores),np.mean(eval_scores))

plt.figure()
plt.hist(eval_scores,np.arange(-200,200+10,10))
plt.grid()
plt.show()

In [ ]:
## use debug mode to check how each agent plays
a_mcts_1.debug_flag = True
run_games(N_iter=1,agents=[a_mcts_2,a_mcts_1,a_mcts_2,a_mcts_1],if_random_game=False,if_display=True,
                  fname='dataset/valid_set_q.p',state_generator=StateExtend)

#### human-AI interaction

In [2]:
%gui asyncio

In [3]:
## load models and agents

deck_size = len(Game().decks)
q_model = QModel(save_dir='model/',deck_size=deck_size)
q_model.restore('dqn-v1/50000')

# a_ai = RandomAgent()
a_ai = RlAgent(q_model)
a_ai.infer_flag = True
# a_ai.debug_flag = True

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model/dqn-v1/50000/model


In [4]:
## start playing

a_ipy = IpyAgent()
# agents = [a_ipy,a_ipy,a_ipy,a_ipy]
agents = [a_ai,a_ipy,a_ai,a_ipy]
env = Env(Game(),DefaultPlayer(),state_generator=StateExtend)
state,info_set,_ = generate_random_game(env,if_display_private=True,if_display_public=False)
task = asyncio.ensure_future(run_single_game_ipy(agents,state,info_set,env,
                                                 if_display_private=True,if_display_public=False))

major suit:♣, major level:3, curr house:0.
--------------------
round:0,player:0 -> ♣5,♣5, curr best ♣5,♣5. player remain: [?],?.
**************************************************
*****   Interactive player: (♦ / ♥ / ♠ / ♣) ({'2': 10, '5': 10})
*****   Game major ♣3. Round best play ♣5,♣5, round total score 20. Current game score 0. Stack ?.
**************************************************
*****   △ Player 0    | this round: ♣5,♣5 | remain cards: ?.
*****   --> Player 1  | this round: ---   | remain cards: ♦2,♦A / ♥4 / ♠A / ♣2,♣A,♥3,♣3,♣3.
*****   Player 2      | this round: ---   | remain cards: ?.
*****   Player 3      | this round: ---   | remain cards: ?.
**************************************************


Dropdown(options=(('Please select from below:', ([], None, False)), ('♣3,♣3', ((19, 19), [(2, 19, (19, 19))], …

round:0,player:1 -> ♣3,♣3, curr best ♣3,♣3. player remain: [?],?.
round:0,player:2 -> ♣4,♣A, curr best ♣3,♣3. player remain: [?],?.
**************************************************
*****   Interactive player: (♦ / ♥ / ♠ / ♣) ({'2': 10, '5': 10})
*****   Game major ♣3. Round best play ♣3,♣3, round total score 20. Current game score 0. Stack ?.
**************************************************
*****   △ Player 0    | this round: ♣5,♣5 | remain cards: ?.
*****   Player 1      | this round: ♣3,♣3 | remain cards: ?.
*****   Player 2      | this round: ♣4,♣A | remain cards: ?.
*****   --> Player 3  | this round: ---   | remain cards: ♦4,♦5 / ♥A / --- / ♣2,♦3,♦3,♠3,☆,☆☆.
**************************************************


Dropdown(options=(('Please select from below:', ([], None, False)), ('♦3,♦3', ((16, 16), [(2, 16, (16, 16))], …

round:0,player:3 -> ♦3,♦3, curr best ♣3,♣3. player remain: [?],?.
round:1,current score:20, eval score:-20.
--------------------
**************************************************
*****   Interactive player: (♦ / ♥ / ♠ / ♣) ({'2': 10, '5': 10})
*****   Game major ♣3. Round best play ---, round total score 0. Current game score 20. Stack ?.
**************************************************
*****   --> Player 1  | this round: --- | remain cards: ♦2,♦A / ♥4 / ♠A / ♣2,♣A,♥3.
*****   Player 2      | this round: --- | remain cards: ?.
*****   Player 3      | this round: --- | remain cards: ?.
*****   △ Player 0    | this round: --- | remain cards: ?.
**************************************************


...

round:5,player:3 -> ♣2,♠3, curr best ♠4,♠4. player remain: [?],?.
round:6,current score:40, eval score:20.
--------------------
round:6,player:0 -> ♣4, curr best ♣4. player remain: [?],?.
**************************************************
*****   Interactive player: (♦ / ♥ / ♠ / ♣) ({'2': 10, '5': 10})
*****   Game major ♣3. Round best play ♣4, round total score 0. Current game score 40. Stack ?.
**************************************************
*****   △ Player 0    | this round: ♣4  | remain cards: ?.
*****   --> Player 1  | this round: --- | remain cards: --- / --- / --- / ♣2.
*****   Player 2      | this round: --- | remain cards: ?.
*****   Player 3      | this round: --- | remain cards: ?.
**************************************************


Dropdown(options=(('Please select from below:', ([], None, False)), ('♣2', ((12,), [(1, 12, (12,))], True))), …

round:6,player:1 -> ♣2, curr best ♣4. player remain: [?],?.
round:6,player:2 -> ♥2, curr best ♣4. player remain: [?],?.
**************************************************
*****   Interactive player: (♦ / ♥ / ♠ / ♣) ({'2': 10, '5': 10})
*****   Game major ♣3. Round best play ♣4, round total score 20. Current game score 40. Stack ?.
**************************************************
*****   △ Player 0    | this round: ♣4  | remain cards: ?.
*****   Player 1      | this round: ♣2  | remain cards: ?.
*****   Player 2      | this round: ♥2  | remain cards: ?.
*****   --> Player 3  | this round: --- | remain cards: --- / --- / --- / ☆.
**************************************************


Dropdown(options=(('Please select from below:', ([], None, False)), ('☆', ((20,), [(1, 20, (20,))], True))), v…

round:6,player:3 -> ☆, curr best ☆. player remain: [?],?.
round:7,current score:140, eval score:-80.
--------------------
